In [4]:
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import pprint
import os, re, json

from fake_useragent import UserAgent
ua = UserAgent(use_external_data=True)


prefix = 'https://www.gutenberg.org'
url = prefix + "/browse/languages/zh" 

listData = []

my_headers = {
    'user-agent': ua.random
}

folderPath = 'gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [5]:
listData.clear()

def getMainLinks():
    
    res = req.get(url, headers = my_headers)
    soup = bs(res.text, 'lxml')
#     print(len(soup.select('li.pgdbetext a[href]')))
    for a in soup.select('li.pgdbetext a[href]'):
        listData.append({
            'title': a.get_text(),
            'link': prefix + a['href'],
            'sub' : []
        })

def getSubLinks():
    for i in range(len(listData)):
        res = req.get(listData[i]['link'], headers = my_headers)
        soup = bs(res.text, "lxml")
        a_elms = soup.select('div[rel="dcterms:hasFormat"] a[type="text/plain; charset=utf-8"]')
    
        if len(a_elms) > 0:
            for a in a_elms:
                listData[i]['sub'].append({
                    'sub_title': a.get_text(),
                    'sub_link': prefix + a['href']
                })

# json
def saveJson():
    fp = open('gutenberg.json', 'w', encoding='utf-8')
    fp.write( json.dumps(listData, ensure_ascii=False, indent=4))
    fp.close()
       
# txt
def writeTxt():
    fp = open('gutenberg.json', 'r', encoding='utf-8')
    strJson = fp.read()
    fp.close()
    
    # 讀全部內容
    listResult = json.loads(strJson)
    for i in range(len(listData)):
        for j in range(len(listResult[i]['sub'])):
            res = req.get(listResult[i]['sub'][j]['sub_link'], headers = my_headers)
            res.encoding = 'utf-8'
            soup = bs(res.text, "lxml")
            strContent = str(res.text)

            
            regex = re.compile(r'[\u4E00-\u9FFF\uFF00-\uFFFF]+')
            arr = re.findall(regex, strContent)
            strContent = ' '.join(arr)
             
            
            try:
                fileName = f'{listResult[i]["title"]}.txt'
                fp = open(f"{folderPath}/{fileName}", "w", encoding="utf-8")
                fp.write(strContent)
                fp.close()
                
            except OSError:
                pass

In [6]:
'''主程式'''
if __name__ == "__main__":
    getMainLinks()
    getSubLinks()
    saveJson()
    writeTxt()